#  The Static and Shared Library with GCC



Change to the shared library working directory to **./demo**

In [1]:
%cd demo

J:\SEU\SEECW\SE\SEES\notebook\demo


##  The Library 


Library files are a collection of **precompiled functions** that are written according to the principle of reusability. 


### Static  Library 

Static libraries are **directly linked into** the program at compile time. 

A program using a static library takes copies of the code that it uses from the static library and makes it part of the program. 


### Shared  Library 

For a shared library, the compiler/linker checks that the names you link with exist in the library when the application is built, **but doesn't move their code into the application**. At run time, the shared library must be available.

Before the executable starts running, **the operating system loads the machine code needed for the external functions** - a process known as **dynamic linking.** 

![library](./img/library.png) 

### The advantages and disadvantages in each method

There are two powerful ideas for modular programming with C/C++ programs, 

* Modular organization of `sources`:  organize program sources into a source tree

* Modular organization of `object files`:  combine object files into libraries

**The advantages of using library**

1. Modular development

2. Reusability

3. Maintainability

**Shared libraries**

* reduce the amount of code that is duplicated in each program that makes use of the library, keeping **binaries small**. 

* allows you to replace the shared object with one that is functionally equivalent, but may have added performance benefits **without needing to recompile** the program that makes use of it. 

Shared libraries will, however have a small additional cost for the execution of the functions as well as a run-time loading cost as all the symbols in the library need to be connected to the things they use

**Static libraries**

* increase the overall size of the binary, but it means that you don't need to carry along a copy of the library that is being used. As the code is connected at compile time there are not any additional run-time loading costs. The code is simply there.





**GCC assumes that all libraries**
   
* `start` with **lib**

* Static libraries: `end` with `.a`

* Shared libraries: `end`  with `.dll`(windows) or `.so`(Linux)，




## 1 The shared library



### 1.1 The source code of library 
The  shared library we will build consist of a single source file: 

* `statistics.c/h`


In [2]:
%%file ./include/statistics.h
#ifndef STATISTICS_H
#define STATISTICS_H

double  mean(double data[], int size);

#endif


Writing ./include/statistics.h


In [3]:
%%file ./src/statistics.c

#include "statistics.h"

double  mean(double data[], int size)
{
  /* Compute the arithmetic mean of a dataset using the recurrence relation 
     mean_(n) = mean(n-1) + (data[n] - mean(n-1))/(n+1)   */

  double mean = 0;
  for(int  i = 0; i < size; i++)
  {
      mean += (data[i] - mean) / (i + 1);
   }
  return mean;
}

Writing ./src/statistics.c



### 1.2 Building the shared library



#### Step 1: Compiling with Position Independent Code

We compile the C file with `Position Independent Code( PIC )` 


* **`-fPIC`** : stands for `Position Independent Code`(位置无关代码)
   
   the generated machine code is `not dependent` on being located at a `specific address` in order to `work`.
   
   Position-independent code can be `executed` at `any memory address`
     

In [4]:
!gcc -c -O3 -Wall -fPIC  -o  ./obj/statistics.o  ./src/statistics.c -I./include/

* `-c`: compile into object file with default name : *.o.

      By default, the object file has the same name as the source file with extension of ".o" 
  
  
* `-O3`: Optimize yet more.

      turns on all optimizations specified by -O2 and also turns on the -finline-functions, -fweb, -frename-registers and -funswitch-loops optionsturns  on  all  optimizations   
     
  
* `-Wall`: prints "all"  compiler's warning message. 

      This option should always be used, in order to generate better code.

#### Step 2: Creating a shared library from an object file

* -shared : creating a shared library

In [5]:
!gcc -shared -o  ./bin/libstat.dll ./obj/statistics.o

**Makefile of Building the shared library**

In [1]:
%%file ./makestatso.mk

CC=gcc
CFLAGS=-O3 -Wall -fPIC 

SRCDIR=./src/
OBJDIR=./obj/
BINDIR=./bin/
INCDIR=./include/

SRCS=$(SRCDIR)statistics.c
OBJS=$(OBJDIR)statistics.o

# Windows
LIBNAME=libstat.dll

all:  $(LIBNAME)

$(LIBNAME):$(OBJS)
	 $(CC) -shared -o $(BINDIR)$@ $^
    
$(OBJS): $(SRCS)
	 $(CC) -c $(CFLAGS) -o $@ $^ -I$(INCDIR)    


Writing ./makestatso.mk


In [7]:
!make -f makestatso.mk

gcc -shared -o ./bin/libstat.dll obj/statistics.o


In [8]:
!dir .\bin\libstat*.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEECW\SE\SEES\notebook\demo\bin 的目录

2022/06/30  23:18            99,244 libstat.dll
               1 个文件         99,244 字节
               0 个目录 95,108,272,128 可用字节


In [6]:
%%file ./makestatso_nonobj.mk

CC=gcc
CFLAGS=-shared -O3 -Wall -fPIC 

SRCDIR=./src/
BINDIR=./bin/
INCDIR=./include/

SRCS=$(SRCDIR)statistics.c

# Windows
LIBNAME=libstat.dll

all:  $(LIBNAME)

$(LIBNAME):$(SRCS)
	 $(CC)  -o $(BINDIR)$@ $(CFLAGS) $^ -I$(INCDIR)   


Overwriting ./makestatso_nonobj.mk


In [7]:
!make -f makestatso_nonobj.mk

gcc  -o ./bin/libstat.dll -shared -O3 -Wall -fPIC  src/statistics.c -I./include/   


#### Step3: Using the shared library

##### Header Files and Libraries 

* `Header File`: When compiling the program, the **compiler** needs the **header** files to compile the source codes;

* `libraries`: the **linker** needs the **libraries** to resolve external references from other object files or libraries. 

The `compiler` and `linker` will not find the `headers/libraries` unless you set **the appropriate options**

* **1 Searching for Header Files**

   **`-Idir`:** The include-paths are specified via **-Idir** option (`uppercase` 'I' followed by the directory path. 
   
   
* **2 Searching for libraries Files**

   **`-Ldir`**: The library-path is specified via **-Ldir** option (`uppercase` 'L' followed by the directory path）. 


* **3 Linking the library**

   **`-lname`**: Link with the library name **without** the `lib` prefix and the extensions.
 


We use the **same** code ` ./src/statdemo.c` to demonstrates calling the shared library 



**Makefile**

In [8]:
%%file ./makestatsodemo.mk

SRCDIR=./src/
BINDIR=./bin/
INCDIR=./include/

all: statdemoso

statdemoso: $(SRCDIR)statdemo.c
	gcc -o $(BINDIR)$@ $^ -L$(BINDIR) -lstat -I$(INCDIR)
    
run:
	./bin/statdemoso    

Overwriting ./makestatsodemo.mk


In [9]:
!make -f makestatsodemo.mk

gcc -o ./bin/statdemoso src/statdemo.c -L./bin/ -lstat -I./include/


In [10]:
!dir .\bin\statdemo*.*

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEECW\SE\SEES\notebook\demo\bin 的目录

2022/06/30  23:46           368,178 statdemoso.exe
               1 个文件        368,178 字节
               0 个目录 95,105,593,344 可用字节


In [29]:
!make -f makestatsodemo.mk run

./bin/statdemoso    
mean is 4.400000


#### Step4 Automatic building all


In [11]:
%%file ./makefile

all:
	make -f makestatso_nonobj.mk
	make -f makestatsodemo.mk
	make -f makestatsodemo.mk run

Writing ./makefile


In [12]:
!make 

make -f makestatso_nonobj.mk
make[1]: Entering directory 'J:/SEU/SEECW/SE/SEES/notebook/demo'
gcc  -o ./bin/libstat.dll -shared -O3 -Wall -fPIC  src/statistics.c -I./include/   
make[1]: Leaving directory 'J:/SEU/SEECW/SE/SEES/notebook/demo'
make -f makestatsodemo.mk
make[1]: Entering directory 'J:/SEU/SEECW/SE/SEES/notebook/demo'
gcc -o ./bin/statdemoso src/statdemo.c -L./bin/ -lstat -I./include/
make[1]: Leaving directory 'J:/SEU/SEECW/SE/SEES/notebook/demo'
make -f makestatsodemo.mk run
make[1]: Entering directory 'J:/SEU/SEECW/SE/SEES/notebook/demo'
./bin/statdemoso    
mean is 4.400000
make[1]: Leaving directory 'J:/SEU/SEECW/SE/SEES/notebook/demo'


## 2 Building  a Shared Library with Multi-source

### 2.1  The multi-source files
The shared library we will build consist of the multi-source files

* statistics.c/h

* number.c/h


In [17]:
%%file ./include/number.h

#ifndef NUMBER_H
#define NUMBER_H

 
int factorial(int n);


#endif

Writing ./include/number.h


In [18]:
%%file ./src/number.c

#include "number.h"

// The factorial of a positive integer n, denoted by n!, 
//    is the product of all positive integers less than or equal to n. 
//  For example,5!=5*4*3*2*1=120
//  The value of 0! is 1 
int factorial(int n)
{
    if (n == 0 ) {
        return 1;
    }
    else 
    {
        return n * factorial(n - 1);
    }
}


Writing ./src/number.c


### 2.2 Building  into One Shared Library

**Makefile**

```make
SRCS=$(SRCDIR)statistics.c \
    $(SRCDIR)number.c 
```    

In [37]:
%%file ./makelibmaths.mk

CC=gcc
CFLAGS=-shared -O3 -Wall -fPIC

SRCDIR= ./src/
BINDIR= ./bin/
INCDIR= ./include/

LIB=libmaths.dll

SRCS=$(SRCDIR)statistics.c \
    $(SRCDIR)number.c 

all:$(LIB)
    
$(LIB): $(SRCS)  
	$(CC) $(CFLAGS) -o $(BINDIR)$@ $^  -I$(INCDIR)

Overwriting ./makelibmaths.mk


In [38]:
!make -f makelibmaths.mk

gcc -shared -O3 -Wall -fPIC -o ./bin/libmaths.dll src/statistics.c src/number.c  -I./include/


In [39]:
!dir .\bin\libm*.dll

 驱动器 J 中的卷是 cmh
 卷的序列号是 9C25-3306

 J:\SEU\SEECW\SE\SEES\notebook\demo\bin 的目录

2022/06/30  22:41            99,542 libmaths.dll
               1 个文件         99,542 字节
               0 个目录 95,112,318,976 可用字节


### 2.3 Building a client executable 

The code demonstrates calling the shared library's functions:

#### 2.3.1 C 

In [40]:
%%file ./src/demomath.c

#include <stdio.h> 
#include "statistics.h"
#include "number.h"

int main() {
    
     double a[] = {8, 4, 5, 3, 2};
     int len= sizeof(a)/sizeof(*a);
     printf("mena is %f\n", mean(a,  len)); 
    
     int n =5;
     printf("the factorial of  %d is %d\n",n,factorial(n));  // 5!=120
     return 0;
}

Overwriting ./src/demomath.c


In [41]:
!gcc -o  ./bin/demomath ./src/demomath.c  -L./bin/ -lmaths -I./include

In [42]:
!.\bin\demomath

mena is 4.400000
the factorial of  5 is 120


#### 2.3.2 CPP

C++ has a special keyword to declare a function with C bindings: **extern "C"**

A function declared as **extern "C"** uses the function name as symbol name, just as a C function

```cpp
#ifdef __cplusplus
extern "C"
{
#endif

// all of your legacy C code here
    
#ifdef __cplusplus
}
#endif

```

The Modified Header Files

In [43]:
%%file ./include/statistics.h
#ifndef STATISTICS_H
#define STATISTICS_H

#ifdef __cplusplus
extern "C"
{
#endif
    
double  mean(double data[], int size);

#ifdef __cplusplus

}
#endif

#endif


Overwriting ./include/statistics.h


In [44]:
%%file ./include/number.h

#ifndef NUMBER_H
#define NUMBER_H


#ifdef __cplusplus
extern "C"
{
#endif
    
int factorial(int n);


#ifdef __cplusplus
}
#endif

#endif

Overwriting ./include/number.h


In [45]:
%%file ./src/demomath.cpp

#include <iostream>

#include "statistics.h"
#include "number.h"

using namespace std;

int main() {
    
     double a[] = {8, 4, 5, 3, 2};
     int len= sizeof(a)/sizeof(*a);
     cout<<"mean is "<<mean(a,  len)<<endl; 
    
     int n =5;
     cout<<"the factorial of "s<<n<<" is "<<factorial(n)<<endl;  // 5!=120
     return 0;
}

Writing ./src/demomath.cpp


In [46]:
!g++ -o  ./bin/demomath ./src/demomath.cpp  -L./bin/ -lmaths -I./include

In [47]:
!.\bin\demomath

mean is 4.4
the factorial of 5 is 120


## 3 The C/C++ Caller of CoolProp Library in Windows

* [Official Doc: CoolProp Shared Library](http://www.coolprop.org/coolprop/wrappers/SharedLibrary/index.html#)


change to **./notebook**

In [ ]:
%pwd

In [ ]:
%cd ../


**The working dir:`/notebook/coolpropdemo`**

![CoolPropdemo](./img/vcr/coolpropdemo.jpg)


**The Windows Shared Library**

1. The official Windows Shared Library builded Microsoft Visuall C++: `CoolProp.dll`

2. The Shared Library builded with TDM-GCC-64: `libCoolProp.dll`

The example use a TDM-GCC-64's dll: **libCoolProp.dll** 

> **Note:** the `coolpropdemo` have demo codes for **Windows** and **Linux**
> 
>  the details about Using The CoolProp shared library under Linux in **Unit8-2-Programming_On_Linux**


#### C Caller

* -DCOOLPROP_LIB 

In [ ]:
%%file ./coolpropdemo/main.c

/* 
The example use the shared library of CoolProp in c

g++ -DCOOLPROP_LIB -o ./bin/main   main.c -I./include -L./bin -lCoolProp
./bin/main

*/

#include "CoolPropLib.h"
#include <stdio.h>

int main() {
     double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
     printf("%f\n", value); 
     return 0;
}


Compiling

In [ ]:
!g++  -DCOOLPROP_LIB -o ./coolpropdemo/bin/main  ./coolpropdemo/main.c -I./coolpropdemo/include -L./coolpropdemo/bin -lCoolProp

Run

In [ ]:
!.\coolpropdemo\bin\main

#### C++ Caller

In [ ]:
%%file ./coolpropdemo/demo.cpp
/* 
The example use the shared library of CoolProp in c++

g++ -DCOOLPROP_LIB -o ./bin/demo  demo.cpp -I./include -L./bin -lCoolProp

./bin/demo

*/

#include <iostream>
#include <iomanip>
#include "CoolPropLib.h"

int main()
{
    double value=PropsSI("P","T",273.15+0,"Q",0,"R134a");
    std::cout << std::setiosflags(std::ios::fixed)  << value << std::endl;
    return 1;
}

Compiling

In [ ]:
!g++ -DCOOLPROP_LIB -o ./coolpropdemo/bin/demo  ./coolpropdemo/demo.cpp -I./coolpropdemo/include -L./coolpropdemo/bin -lCoolProp

Run

In [ ]:
!.\coolpropdemo\bin\demo